In [2]:
# Load datasets from Hugging Face
from datasets import load_dataset

marketing_data = load_dataset("RafaM97/marketing_social_media")['train']

tweet_eval = load_dataset('tweet_eval', 'sentiment')['train']

# Convert to pandas for easier handling
marketing_df = pd.DataFrame(marketing_data)
tweet_df = pd.DataFrame(tweet_eval)

README.md:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

marketing_social_media_dataset_v2.json:   0%|          | 0.00/499k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/689 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.78M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/901k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/167k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [1]:
!pip install nltk transformers datasets textstat
import pandas as pd
import numpy as np
import random
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from transformers import pipeline
from nltk.stem import WordNetLemmatizer
import re
from scipy.sparse import hstack
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from transformers import TrainerCallback
from datasets import load_dataset
from tqdm import tqdm
import time
import os
import torch  
import uuid
%matplotlib inline
nltk.download('vader_lexicon')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.4 MB/s eta 0:00:00:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; plat

2025-05-14 11:33:46.460961: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747222426.647563      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747222426.699235      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
marketing_df

,instruction,input,response
0,Develop a social media campaign to increase br...,Company: GreenCycle Clothing\nTarget Audience:...,"""Revolutionize Your Wardrobe"" campaign, levera..."
1,Create a content calendar for a B2B software c...,Company: NovaTech Solutions\nTarget Audience: ...,"""AI Insights"" series, covering topics like AI ..."
2,Design a targeted email campaign to re-engage ...,Company: FitZone\nTarget Audience: Inactive su...,"""Get Moving Again"" campaign, featuring a 3-ema..."
3,Create a social media campaign to increase bra...,"Company: GreenThreads, a fashion brand using e...",Develop a social media campaign highlighting G...
4,Design an email marketing campaign to nurture ...,"Company: Proxima, a SaaS provider for marketin...",Create a 5-email nurture sequence: Introductio...
...,...,...,...
684,Write a script for a social media video promot...,Company: Green Bites\nTarget Audience: Health-...,Create a 60-second video showcasing the servic...
685,Develop a content calendar for a B2B software ...,Company: Tech Solutions\nTarget Audience: Busi...,Create a content calendar focusing on topics s...
686,Create a social media campaign to increase bra...,"Company: GreenThreads, a sustainable fashion b...","""Threads for a Greener Tomorrow"" campaign, foc..."
687,Develop a content marketing strategy for a new...,"Company: FitGenius, an AI-powered fitness app;...","""Fitness on Demand"" content hub, featuring bit..."


In [4]:
tweet_df

,text,label
0,"""QT @user In the original draft of the 7th boo...",2
1,"""Ben Smith / Smith (concussion) remains out of...",1
2,Sorry bout the stream last night I crashed out...,1
3,Chase Headley's RBI double in the 8th inning o...,1
4,@user Alciato: Bee will invest 150 million in ...,2
...,...,...
45610,"@user \""""So amazing to have the beautiful Lady...",2
45611,"9 September has arrived, which means Apple's n...",2
45612,Leeds 1-1 Sheff Wed. Giuseppe Bellusci securin...,2
45613,@user no I'm in hilton head till the 8th lol g...,1


In [5]:

# Custom callback for progress tracking
class ProgressCallback(TrainerCallback):
    def __init__(self, total_steps):
        self.total_steps = total_steps
        self.start_time = None
        self.pbar = None
        self.last_loss = None

    def on_train_begin(self, args, state, control, **kwargs):
        self.start_time = time.time()
        self.pbar = tqdm(total=self.total_steps, desc="Training Progress")

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and 'loss' in logs:
            self.last_loss = logs['loss']

    def on_step_end(self, args, state, control, **kwargs):
        self.pbar.update(1)
        percent_complete = (state.global_step / self.total_steps) * 100
        elapsed_time = time.time() - self.start_time
        avg_time_per_step = elapsed_time / state.global_step if state.global_step > 0 else 0
        remaining_steps = self.total_steps - state.global_step
        estimated_time_remaining = remaining_steps * avg_time_per_step
        self.pbar.set_postfix({
            'Step': f'{state.global_step}/{self.total_steps}',
            'Progress': f'{percent_complete:.1f}%',
            'Loss': f'{self.last_loss:.4f}' if self.last_loss is not None else 'N/A',
            'ETA': f'{estimated_time_remaining/60:.1f} min'
        })

    def on_epoch_end(self, args, state, control, **kwargs):
        print(f"Epoch {state.epoch} completed")

    def on_train_end(self, args, state, control, **kwargs):
        self.pbar.close()

In [6]:
file_path = '/kaggle/working/intermediate_ads.csv'
if os.path.exists(file_path):
    print(f"Loading existing dataset from {file_path}")
    df = pd.read_csv(file_path)

else:
    print("Generating new synthetic ad campaign dataset")
    n_records = 2000
    data = {
        'Campaign_ID': [f'C{str(i).zfill(3)}' for i in range(n_records)],
        'Ad_Text': [''] * n_records,  # To be filled
        'Product_Description': [''] * n_records,  # To be filled
        'Category': np.random.choice(['Apparel', 'Electronics', 'Food & Beverage', 'Beauty'], n_records),
        'Platform': np.random.choice(['Twitter', 'Instagram', 'Facebook'], n_records),
        'Date': pd.date_range(start='2025-01-01', end='2025-12-31', periods=n_records),
        'Sentiment_Score': [0.0] * n_records,  # To be filled
        'Sentiment_Label': [''] * n_records,  # To be filled
        'Comments': [[]] * n_records,  # To be filled
        'Avg_Comment_Sentiment': [0.0] * n_records,  # To be filled
        'Impressions': np.random.randint(10000, 1000000, n_records),
        'Money_Spent': np.random.randint(1000, 100000, n_records),
    
        'CTR': [0.0] * n_records,  # Initialize Click-Through Rate
        'ROI': [0.0] * n_records,  # Initialize Return on Investment
        'Goods_Sold': [0] * n_records,  # Initialize number of goods sold
        'Likes': [0] * n_records,  # Initialize number of likes
        'Retweets': [0] * n_records,  # Initialize number of retweets
        'Conversions': [0] * n_records,
        'Conversion_Rate': [0.0] * n_records,
        'Cost_Per_Conversion': [0.0] * n_records,
        'Comment_Count': [0] * n_records,
        'Engagement_Rate': [0.0] * n_records,
        'Brand_Sentiment_Lift': [0.0] * n_records,
        'Share_of_Voice': [0.0] * n_records
    }
    df = pd.DataFrame(data)

    
    
    # Fine-tune GPT-2 for ads
    # Save marketing instructions
    with open('marketing_instructions.txt', 'w') as f:
        for instr in marketing_df['instruction']:
            f.write(instr + '\n')
    
    # Load dataset using Datasets library
    dataset = load_dataset('text', data_files={'train': 'marketing_instructions.txt'})['train']
    
    # Tokenize dataset
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token  # Set pad token
    
    def tokenize_function(examples):
        return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)
    
    tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['text'])
    tokenized_dataset.set_format('torch', columns=['input_ids', 'attention_mask'])
    
    # Initialize model and data collator
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    
    # Calculate total training steps
    batch_size = 4
    num_epochs = 2
    total_steps = len(tokenized_dataset) // batch_size * num_epochs
    
    training_args = TrainingArguments(
        output_dir='./gpt2_marketing',
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        save_steps=500,
        save_total_limit=2,
        logging_dir='./logs',
        logging_steps=10,  # Increased frequency for better loss tracking
        report_to="none"
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator,
        callbacks=[ProgressCallback(total_steps)]
    )
    trainer.train()
    
    # Explicitly save the final model
    trainer.save_model('./gpt2_marketing')
    tokenizer.save_pretrained('./gpt2_marketing')
    
    # Debug: Check directory contents
    print("Contents of ./gpt2_marketing:", os.listdir('./gpt2_marketing'))
    
    # Find the latest checkpoint if exists
    checkpoint_dir = './gpt2_marketing'
    if os.path.exists(checkpoint_dir):
        checkpoints = [d for d in os.listdir(checkpoint_dir) if d.startswith('checkpoint-')]
        if checkpoints:
            latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('-')[1]))
            checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)
            print(f"Using latest checkpoint: {checkpoint_path}")
        else:
            checkpoint_path = checkpoint_dir
    else:
        print("No checkpoints found, using base model directory")
        checkpoint_path = checkpoint_dir
    
    # Initialize text generation pipeline
    try:
        generator = pipeline('text-generation', model=checkpoint_path, tokenizer=tokenizer)
    except Exception as e:
        print(f"Failed to load model from {checkpoint_path}: {e}")
        print("Falling back to pre-trained 'gpt2' model")
        generator = pipeline('text-generation', model='gpt2', tokenizer=tokenizer)

        
    # Clear GPU memory
    torch.cuda.empty_cache()
    
    # Generate ads with progress bar and batch processing
    def generate_texts(categories, prompt_type='slogan', batch_size=8):
        prompts = [f"Create a {'slogan' if prompt_type == 'slogan' else 'product description'} for a {cat} campaign: " for cat in categories]
        outputs = []
        for i in tqdm(range(0, len(prompts), batch_size), desc=f"Generating {prompt_type}s"):
            batch_prompts = prompts[i:i + batch_size]
            batch_outputs = generator(batch_prompts, max_length=150 if prompt_type == 'description' else 70, num_return_sequences=1, truncation=True)
            outputs.extend([out[0]['generated_text'].replace(prompt, '').strip() for out, prompt in zip(batch_outputs, batch_prompts)])
        return outputs
    
    # Apply to dataset with logging
    print("Generating Ad_Text...")
    df['Ad_Text'] = generate_texts(df['Category'].tolist(), prompt_type='slogan')
    print("Generating Product_Description...")
    df['Product_Description'] = generate_texts(df['Category'].tolist(), prompt_type='description')
    
    print("Computing sentiment analysis...")
    def compute_sentiment(text):
        return TextBlob(str(text)).sentiment.polarity
    
    # Combine Ad_Text and Product_Description for sentiment
    df['Sentiment_Score'] = (df['Ad_Text'].apply(compute_sentiment) + df['Product_Description'].apply(compute_sentiment)) / 2
    
    # Assign Sentiment_Label based on Sentiment_Score
    def assign_sentiment_label(score):
        if score > 0.1:
            return 'Positive'
        elif score < -0.1:
            return 'Negative'
        else:
            return 'Neutral'
    
    df['Sentiment_Label'] = df['Sentiment_Score'].apply(assign_sentiment_label)
    
    df['Conversions'] = [int(i * c * np.random.uniform(0.05, 0.15)) for i, c in zip(df['Impressions'], df['CTR'])]
    df['Goods_Sold'] = [int(c * np.random.uniform(0.5, 2)) for c in df['Conversions']]
    df['ROI'] = [min(10, max(1, s * 2 + a * 1 + np.random.normal(3, 1))) for s, a in zip(df['Sentiment_Score'], df['Avg_Comment_Sentiment'])]
    df['Likes'] = [int(i * c * np.random.uniform(0.05, 0.2)) if s > 0 else int(i * c * np.random.uniform(0.01, 0.05)) for i, c, s in zip(df['Impressions'], df['CTR'], df['Avg_Comment_Sentiment'])]
    df['Retweets'] = [int(l * np.random.uniform(0.1, 0.5)) for l in df['Likes']]
    df['Conversion_Rate'] = df['Conversions'] / df['Impressions']
    df['Cost_Per_Conversion'] = df['Money_Spent'] / df['Conversions'].replace(0, 1)
    df['Comment_Count'] = df['Comment_Count']  # Already set
    df['Engagement_Rate'] = (df['Likes'] + df['Retweets'] + df['Comment_Count']) / df['Impressions']
    df['Brand_Sentiment_Lift'] = df['Avg_Comment_Sentiment'] - 0.0  # Neutral baseline
    category_impressions = df.groupby('Category')['Impressions'].sum()
    df['Share_of_Voice'] = df.apply(lambda row: row['Impressions'] / category_impressions[row['Category']], axis=1)
    
    # Debug sentiment values
    print("Sentiment_Score statistics:")
    print(df['Sentiment_Score'].describe())
    print("Sentiment_Label distribution:")
    print(df['Sentiment_Label'].value_counts())
    print("Avg_Comment_Sentiment statistics:")
    print(df['Avg_Comment_Sentiment'].describe())


    # Save DataFrame
    csv_path = '/kaggle/working/intermediate_ads.csv'
    df.to_csv(csv_path, index=False)
    print(f"DataFrame saved to {csv_path}")
df

Loading existing dataset from /kaggle/working/intermediate_ads.csv


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Campaign_ID,Ad_Text,Product_Description,Category,Platform,Date,Sentiment_Score,Sentiment_Label,Comments,Avg_Comment_Sentiment,...,Goods_Sold,Likes,Retweets,Conversions,Conversion_Rate,Cost_Per_Conversion,Comment_Count,Engagement_Rate,Brand_Sentiment_Lift,Share_of_Voice
0,C000,"""Breakfast is a healthy event for all of us. E...",click the image below to launch a content cale...,Food & Beverage,Facebook,2025-01-01 00:00:00.000000000,0.0,NaN,[],0.0,...,796,263,101,530,0.000582,123.181132,0,0.000399,0.0,0.003642
1,C001,"""Make a difference!"". The product description ...","""Learn to drive sales for a health and wellnes...",Beauty,Instagram,2025-01-01 04:22:12.666333166,0.0,NaN,[],0.0,...,113,65,16,67,0.000147,1131.970149,0,0.000178,0.0,0.001847
2,C002,"""Don't judge a product by its product descript...",make an email marketing campaign targeting you...,Apparel,Facebook,2025-01-01 08:44:25.332666333,0.0,NaN,[],0.0,...,453,150,69,239,0.000546,66.560669,0,0.000500,0.0,0.001551
3,C003,"""Biscuit-free! It's called a freebie contest...",use their social media influencer strategy to ...,Food & Beverage,Facebook,2025-01-01 13:06:37.998999499,0.0,NaN,[],0.0,...,125,585,189,237,0.000351,395.008439,0,0.001146,0.0,0.002699
4,C004,~~Please support this awesome eco-friendly pro...,click here to read about a sustainable food de...,Food & Beverage,Twitter,2025-01-01 17:28:50.665332666,0.0,NaN,[],0.0,...,141,44,12,194,0.000729,266.113402,0,0.000211,0.0,0.001063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,C1995,"""Make a difference with fashion. Start a fashi...","""To promote a new eco-friendly fashion brand's...",Apparel,Facebook,2025-12-30 06:31:09.334667336,0.0,NaN,[],0.0,...,409,546,260,453,0.000545,127.145695,0,0.000969,0.0,0.002947
1996,C1996,"""Think big and act as an influencer for a new ...",Introducing an AI-powered social media campaig...,Electronics,Instagram,2025-12-30 10:53:22.001000500,0.0,NaN,[],0.0,...,279,244,51,225,0.000453,205.386667,0,0.000594,0.0,0.001998
1997,C1997,"""Inspire a B2B lifestyle brand's Instagram pag...","""Introducing an eco-friendly eco-friendly fash...",Apparel,Facebook,2025-12-30 15:15:34.667333668,0.0,NaN,[],0.0,...,24,11,1,39,0.000457,272.743590,0,0.000141,0.0,0.000302
1998,C1998,"""This isn't about animal welfare. This is a c...","""How can I promote a new food program for a yo...",Food & Beverage,Twitter,2025-12-30 19:37:47.333666836,0.0,NaN,[],0.0,...,133,224,65,100,0.000103,442.580000,0,0.000298,0.0,0.003872


In [7]:
print("Computing sentiment analysis...")
def compute_sentiment(text):
    return TextBlob(str(text)).sentiment.polarity

# Combine Ad_Text and Product_Description for sentiment
df['Sentiment_Score'] = (df['Ad_Text'].apply(compute_sentiment) + df['Product_Description'].apply(compute_sentiment)) / 2

# Assign Sentiment_Label based on Sentiment_Score
def assign_sentiment_label(score):
    if score > 0.1:
        return 'Positive'
    elif score < -0.1:
        return 'Negative'
    else:
        return 'Neutral'

df['Sentiment_Label'] = df['Sentiment_Score'].apply(assign_sentiment_label)

# Placeholder for comment sentiment (assuming comments_df is generated separately)
# If comments_df exists, compute Avg_Comment_Sentiment
# comments_file = '/kaggle/working/comments_df.csv'  # Adjust path if needed
# if os.path.exists(comments_file):
#     print(f"Loading comments from {comments_file}")
#     comments_df = pd.read_csv(comments_file)
#     comments_df['Comment_Sentiment'] = comments_df['Comment_Text'].apply(compute_sentiment)
#     comment_agg = comments_df.groupby('Campaign_ID').agg({
#         'Comment_Sentiment': 'mean',
#         'Comment_Text': ['count', lambda x: ' '.join(x)],
#         'Likes': 'sum'
#     }).reset_index()
#     comment_agg.columns = ['Campaign_ID', 'Avg_Comment_Sentiment', 'Comment_Count', 'Comments', 'Likes_sum']
#     df = df.merge(comment_agg, on='Campaign_ID', how='left')
#     df['Avg_Comment_Sentiment'] = df['Avg_Comment_Sentiment'].fillna(0.0)
#     df['Comment_Count'] = df['Comment_Count'].fillna(0)
#     df['Comments'] = df['Comments'].fillna('')
#     df['Likes_sum'] = df['Likes_sum'].fillna(0)
# else:
#     print("No comments_df found; initializing Avg_Comment_Sentiment with synthetic values")
#     df['Avg_Comment_Sentiment'] = np.random.uniform(-0.3, 0.6, n_records)  # Synthetic values
#     df['Comment_Count'] = np.random.randint(2, 11, n_records)
#     df['Comments'] = df['Comment_Count'].apply(lambda x: ['Sample comment'] * x)
#     df['Likes_sum'] = np.random.randint(10, 317, n_records)

# Update metrics
# df['CTR'] = [
#     min(0.1, max(0.001, 
#         0.05 * s + 0.02 * (l / df['Impressions'].mean()) + 0.03 * a + np.random.normal(0.02, 0.01)
#     )) 
#     for s, l, a in zip(df['Sentiment_Score'], df['Likes_sum'], df['Avg_Comment_Sentiment'])
# ]
df['Conversions'] = [int(i * c * np.random.uniform(0.05, 0.15)) for i, c in zip(df['Impressions'], df['CTR'])]
df['Goods_Sold'] = [int(c * np.random.uniform(0.5, 2)) for c in df['Conversions']]
df['ROI'] = [min(10, max(1, s * 2 + a * 1 + np.random.normal(3, 1))) for s, a in zip(df['Sentiment_Score'], df['Avg_Comment_Sentiment'])]
df['Likes'] = [int(i * c * np.random.uniform(0.05, 0.2)) if s > 0 else int(i * c * np.random.uniform(0.01, 0.05)) for i, c, s in zip(df['Impressions'], df['CTR'], df['Avg_Comment_Sentiment'])]
df['Retweets'] = [int(l * np.random.uniform(0.1, 0.5)) for l in df['Likes']]
df['Conversion_Rate'] = df['Conversions'] / df['Impressions']
df['Cost_Per_Conversion'] = df['Money_Spent'] / df['Conversions'].replace(0, 1)
df['Comment_Count'] = df['Comment_Count']  # Already set
df['Engagement_Rate'] = (df['Likes'] + df['Retweets'] + df['Comment_Count']) / df['Impressions']
df['Brand_Sentiment_Lift'] = df['Avg_Comment_Sentiment'] - 0.0  # Neutral baseline
category_impressions = df.groupby('Category')['Impressions'].sum()
df['Share_of_Voice'] = df.apply(lambda row: row['Impressions'] / category_impressions[row['Category']], axis=1)

# Debug sentiment values
print("Sentiment_Score statistics:")
print(df['Sentiment_Score'].describe())
print("Sentiment_Label distribution:")
print(df['Sentiment_Label'].value_counts())
print("Avg_Comment_Sentiment statistics:")
print(df['Avg_Comment_Sentiment'].describe())


Computing sentiment analysis...
Sentiment_Score statistics:
count    2000.000000
mean        0.201077
std         0.105345
min        -0.210000
25%         0.131234
50%         0.193781
75%         0.264307
max         0.800000
Name: Sentiment_Score, dtype: float64
Sentiment_Label distribution:
Sentiment_Label
Positive    1694
Neutral      301
Negative       5
Name: count, dtype: int64
Avg_Comment_Sentiment statistics:
count    2000.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: Avg_Comment_Sentiment, dtype: float64


In [8]:
df

,Campaign_ID,Ad_Text,Product_Description,Category,Platform,Date,Sentiment_Score,Sentiment_Label,Comments,Avg_Comment_Sentiment,...,Goods_Sold,Likes,Retweets,Conversions,Conversion_Rate,Cost_Per_Conversion,Comment_Count,Engagement_Rate,Brand_Sentiment_Lift,Share_of_Voice
0,C000,"""Breakfast is a healthy event for all of us. E...",click the image below to launch a content cale...,Food & Beverage,Facebook,2025-01-01 00:00:00.000000000,0.233333,Positive,[],0.0,...,1260,389,112,1142,0.001253,57.168126,0,0.000550,0.0,0.003642
1,C001,"""Make a difference!"". The product description ...","""Learn to drive sales for a health and wellnes...",Beauty,Instagram,2025-01-01 04:22:12.666333166,0.092929,Neutral,[],0.0,...,349,44,20,357,0.000785,212.442577,0,0.000141,0.0,0.001847
2,C002,"""Don't judge a product by its product descript...",make an email marketing campaign targeting you...,Apparel,Facebook,2025-01-01 08:44:25.332666333,0.326326,Positive,[],0.0,...,704,223,82,634,0.001448,25.091483,0,0.000697,0.0,0.001551
3,C003,"""Biscuit-free! It's called a freebie contest...",use their social media influencer strategy to ...,Food & Beverage,Facebook,2025-01-01 13:06:37.998999499,0.108838,Positive,[],0.0,...,1445,248,83,1185,0.001754,79.001688,0,0.000490,0.0,0.002699
4,C004,~~Please support this awesome eco-friendly pro...,click here to read about a sustainable food de...,Food & Beverage,Twitter,2025-01-01 17:28:50.665332666,0.098674,Neutral,[],0.0,...,202,84,40,137,0.000515,376.832117,0,0.000466,0.0,0.001063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,C1995,"""Make a difference with fashion. Start a fashi...","""To promote a new eco-friendly fashion brand's...",Apparel,Facebook,2025-12-30 06:31:09.334667336,0.090909,Neutral,[],0.0,...,604,188,46,767,0.000922,75.093872,0,0.000281,0.0,0.002947
1996,C1996,"""Think big and act as an influencer for a new ...",Introducing an AI-powered social media campaig...,Electronics,Instagram,2025-12-30 10:53:22.001000500,0.043669,Neutral,[],0.0,...,269,226,52,347,0.000699,133.175793,0,0.000560,0.0,0.001998
1997,C1997,"""Inspire a B2B lifestyle brand's Instagram pag...","""Introducing an eco-friendly eco-friendly fash...",Apparel,Facebook,2025-12-30 15:15:34.667333668,0.239322,Positive,[],0.0,...,31,18,6,36,0.000422,295.472222,0,0.000281,0.0,0.000302
1998,C1998,"""This isn't about animal welfare. This is a c...","""How can I promote a new food program for a yo...",Food & Beverage,Twitter,2025-12-30 19:37:47.333666836,0.092045,Neutral,[],0.0,...,575,104,45,703,0.000726,62.955903,0,0.000154,0.0,0.003872


In [9]:
df['Ad_Text']

0       "Breakfast is a healthy event for all of us. E...
1       "Make a difference!". The product description ...
2       "Don't judge a product by its product descript...
3       "Biscuit-free!   It's called a freebie contest...
4       ~~Please support this awesome eco-friendly pro...
                              ...                        
1995    "Make a difference with fashion. Start a fashi...
1996    "Think big and act as an influencer for a new ...
1997    "Inspire a B2B lifestyle brand's Instagram pag...
1998    "This isn't about animal welfare.  This is a c...
1999    "Wearable and innovative. Available now for pu...
Name: Ad_Text, Length: 2000, dtype: object

In [10]:
analyzer = SentimentIntensityAnalyzer()

# Fine-tune GPT-2 for comment generation
def prepare_comment_dataset():
    comments = tweet_df['text'].tolist()
    with open('comments.txt', 'w') as f:
        for comment in comments:
            f.write(f"{comment}\n")
    return 'comments.txt'

def fine_tune_gpt2_comments():
    model_name = "gpt2"
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    model = GPT2LMHeadModel.from_pretrained(model_name)
    
    dataset_file = prepare_comment_dataset()
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=dataset_file,
        block_size=128
    )
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )
    
    batch_size = 4
    num_epochs = 2
    total_steps = len(dataset) // batch_size * num_epochs
    
    training_args = TrainingArguments(
        output_dir='./gpt2_comments',
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        save_steps=500,
        save_total_limit=2,
        logging_dir='./logs_comments',
        logging_steps=10,
        report_to="none"
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        data_collator=data_collator,
        callbacks=[ProgressCallback(total_steps)]
    )
    
    trainer.train()
    model.save_pretrained('./gpt2_comments')
    tokenizer.save_pretrained('./gpt2_comments')
    return model, tokenizer

# Load or fine-tune GPT-2 comment model
if os.path.exists('./gpt2_comments'):
    print("Loading fine-tuned GPT-2 comment model")
    comment_model = GPT2LMHeadModel.from_pretrained('./gpt2_comments')
    comment_tokenizer = GPT2Tokenizer.from_pretrained('./gpt2_comments')
else:
    print("Fine-tuning GPT-2 model for comment generation")
    comment_model, comment_tokenizer = fine_tune_gpt2_comments()

# Ensure pad_token_id is set
comment_tokenizer.pad_token = comment_tokenizer.eos_token

comments_file_path = '/kaggle/working/ad_comments.csv'
if os.path.exists(comments_file_path):
    print(f"Loading existing comments dataset from {comments_file_path}")
    comments_df = pd.read_csv(comments_file_path)
else:
    print("Generating new comments dataset")

    # Initialize comment generation pipeline
    checkpoint_dir = './gpt2_comments'
    if os.path.exists(checkpoint_dir):
        checkpoints = [d for d in os.listdir(checkpoint_dir) if d.startswith('checkpoint-')]
        if checkpoints:
            latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('-')[1]))
            checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)
            print(f"Using latest checkpoint for comments: {checkpoint_path}")
        else:
            checkpoint_path = checkpoint_dir
    else:
        print("No checkpoints found for comments, using base model directory")
        checkpoint_path = checkpoint_dir
    
    try:
        comment_generator = pipeline('text-generation', model=checkpoint_path, tokenizer=comment_tokenizer, device=0)
    except Exception as e:
        print(f"Failed to load comment model from {checkpoint_path}: {e}")
        print("Falling back to pre-trained 'gpt2' model")
        comment_generator = pipeline('text-generation', model='gpt2', tokenizer=comment_tokenizer, device=0)
    
    # Function to generate tailored comments in batch
    def generate_comments_batch(ad_text, product_desc, n_positive, n_negative, n_neutral, max_new_tokens=50, max_retries=3):
        prompt = f"Ad: {ad_text}\nDescription: {product_desc}\nComment: "
        total_comments = n_positive + n_negative + n_neutral
        comments = []
        
        # Generate more comments than needed to account for sentiment mismatches
        num_generate = int(total_comments * 1.5)  # 50% extra to reduce retries
        inputs = comment_tokenizer([prompt] * num_generate, return_tensors="pt", padding=True, truncation=True, max_length=512)
        input_ids = inputs['input_ids'].to('cuda')
        attention_mask = inputs['attention_mask'].to('cuda')
        
        outputs = comment_generator.model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            num_return_sequences=1,
            pad_token_id=comment_tokenizer.eos_token_id
        )
        
        generated_comments = [comment_tokenizer.decode(output, skip_special_tokens=True).replace(prompt, '').strip() for output in outputs]
        
        # Categorize comments by sentiment
        positive_comments = []
        negative_comments = []
        neutral_comments = []
        
        for comment in generated_comments:
            sentiment_score = analyzer.polarity_scores(comment)['compound']
            if sentiment_score > 0:
                positive_comments.append((comment, sentiment_score))
            elif sentiment_score < -0.1:
                negative_comments.append((comment, sentiment_score))
            else:
                neutral_comments.append((comment, sentiment_score))
        
        # Collect required number of comments, retrying if necessary
        retries = 0
        while (len(positive_comments) < n_positive or len(negative_comments) < n_negative or len(neutral_comments) < n_neutral) and retries < max_retries:
            additional_needed = max(0, n_positive - len(positive_comments)) + max(0, n_negative - len(negative_comments)) + max(0, n_neutral - len(neutral_comments))
            inputs = comment_tokenizer([prompt] * additional_needed, return_tensors="pt", padding=True, truncation=True, max_length=512)
            input_ids = inputs['input_ids'].to('cuda')
            attention_mask = inputs['attention_mask'].to('cuda')
            
            additional_outputs = comment_generator.model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                top_p=0.9,
                temperature=0.7,
                num_return_sequences=1,
                pad_token_id=comment_tokenizer.eos_token_id
            )
            
            for output in additional_outputs:
                comment = comment_tokenizer.decode(output, skip_special_tokens=True).replace(prompt, '').strip()
                sentiment_score = analyzer.polarity_scores(comment)['compound']
                if sentiment_score > 0 and len(positive_comments) < n_positive:
                    positive_comments.append((comment, sentiment_score))
                elif sentiment_score < -0.1 and len(negative_comments) < n_negative:
                    negative_comments.append((comment, sentiment_score))
                elif -0.1 <= sentiment_score <= 0.1 and len(neutral_comments) < n_neutral:
                    neutral_comments.append((comment, sentiment_score))
            
            retries += 1
        
        # Take required number of comments
        comments.extend([(comment, score, 'Positive') for comment, score in positive_comments[:n_positive]])
        comments.extend([(comment, score, 'Negative') for comment, score in negative_comments[:n_negative]])
        comments.extend([(comment, score, 'Neutral') for comment, score in neutral_comments[:n_neutral]])
        
        # If still not enough comments, fill with neutral or random
        while len(comments) < total_comments:
            comment = generated_comments[len(comments) % len(generated_comments)]
            sentiment_score = analyzer.polarity_scores(comment)['compound']
            sentiment = 'Neutral' if -0.1 <= sentiment_score <= 0.1 else 'Positive' if sentiment_score > 0 else 'Negative'
            comments.append((comment, sentiment_score, sentiment))
        
        return comments[:total_comments]
    
    # Generate comments for each ad
    comments_data = []
    n_comments_per_ad = 10
    sentiment_distribution = {'Positive': 0.6, 'Negative': 0.3, 'Neutral': 0.1}
    
    torch.cuda.empty_cache()
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating comments"):
        ad_text = row['Ad_Text']
        product_desc = row['Product_Description']
        campaign_id = row['Campaign_ID']
        platform = row['Platform']
        
        n_positive = int(n_comments_per_ad * sentiment_distribution['Positive'])
        n_negative = int(n_comments_per_ad * sentiment_distribution['Negative'])
        n_neutral = n_comments_per_ad - n_positive - n_negative
        
        comments = generate_comments_batch(ad_text, product_desc, n_positive, n_negative, n_neutral)
        
        for comment, score, sentiment in comments:
            likes = random.randint(5, 50) if platform != 'Twitter' and sentiment == 'Positive' else random.randint(0, 20) if platform == 'Twitter' and sentiment == 'Positive' else random.randint(0, 10) if sentiment == 'Negative' else random.randint(2, 20)
            engagements = random.randint(10, 100) if platform != 'Twitter' and sentiment == 'Positive' else random.randint(0, 30) if platform == 'Twitter' and sentiment == 'Positive' else random.randint(0, 20) if sentiment == 'Negative' else random.randint(5, 50)
            comments_data.append({
                'Comment_ID': str(uuid.uuid4()),
                'Campaign_ID': campaign_id,
                'Comment_Text': comment,
                'Sentiment': sentiment,
                'Sentiment_Score': score,
                'Likes': likes,
                'Engagements': engagements
            })
    
    # Create comments dataframe and save to CSV
    comments_df = pd.DataFrame(comments_data)
    comments_file_path = '/kaggle/working/ad_comments.csv'
    comments_df.to_csv(comments_file_path, index=False)
    print(f"Comments dataset saved to {comments_file_path}")
    
    # Update ad dataframe with comment counts and average sentiment
    for campaign_id in df['Campaign_ID']:
        campaign_comments = comments_df[comments_df['Campaign_ID'] == campaign_id]
        df.loc[df['Campaign_ID'] == campaign_id, 'Comment_Count'] = len(campaign_comments)
        df.loc[df['Campaign_ID'] == campaign_id, 'Avg_Comment_Sentiment'] = campaign_comments['Sentiment_Score'].mean() if len(campaign_comments) > 0 else 0.0
        df.loc[df['Campaign_ID'] == campaign_id, 'Comments'] = campaign_comments['Comment_Text'].tolist()
    
    # Recalculate metrics dependent on Avg_Comment_Sentiment
    # df['Likes'] = [int(i * c * np.random.uniform(0.05, 0.2)) if s > 0 else int(i * c * np.random.uniform(0.01, 0.05)) for i, c, s in zip(df['Impressions'], df['CTR'], df['Avg_Comment_Sentiment'])]
    # df['Retweets'] = [int(l * np.random.uniform(0.1, 0.5)) for l in df['Likes']]
    # df['Engagement_Rate'] = (df['Likes'] + df['Retweets'] + df['Comment_Count']) / df['Impressions']
    # df['Brand_Sentiment_Lift'] = df['Avg_Comment_Sentiment'] - 0.0
    
    # # Save updated ad dataframe
    # df.to_csv(file_path, index=False)
    # print(f"Updated ad dataset saved to {file_path}")
    
# Display the comments dataframe
comments_df

Loading fine-tuned GPT-2 comment model
Loading existing comments dataset from /kaggle/working/ad_comments.csv


,Comment_ID,Campaign_ID,Comment_Text,Sentiment,Sentiment_Score,Likes,Engagements
0,730366fb-10d7-4e52-b64d-7ce640b67c60,C000,@user I think I may have to start watching the...,Positive,0.9061,24,91
1,09cd6301-6355-4157-8a41-f375290efc7e,C000,I may be the only person who isn't obsessed wi...,Positive,0.9013,37,87
2,97a0d969-92c0-4707-8fdf-1259984d3a8e,C000,I'd be very interested to hear from you if you...,Positive,0.7713,17,29
3,e7761d03-bd56-4b46-940f-f97173685f8e,C000,The #Beltway\u002c the #Beltway is the most im...,Positive,0.2716,28,30
4,9576c258-dd63-4317-b28d-30ee078dd189,C000,You may remember that I wrote about the new Ap...,Positive,0.0067,39,77
...,...,...,...,...,...,...,...
19995,4a2a62a3-333a-40e7-a5cd-201efb73f3fa,C1999,I'm going to the game tomorrow and it's going ...,Positive,0.6249,7,16
19996,a7c03387-ed21-4565-8b11-5af2e0a34be4,C1999,NaN,Neutral,0.0000,2,12
19997,626f25ea-4d53-4237-b321-ae30d380c98b,C1999,NaN,Neutral,0.0000,6,22
19998,bbee3f4f-d463-42c2-afbb-993722461703,C1999,@user @user @user,Neutral,0.0000,18,23


In [11]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from scipy.sparse import hstack
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
import re

# --- Campaign Success Prediction ---
print("Estimating campaign success based on slogan, description, likes, and comments...")

# Debug input data
# print("Columns in df before processing:", df.columns.tolist())
# print("Columns in comments_df:", comments_df.columns.tolist())
# print("Sample Ad_Text:", df['Ad_Text'].head().tolist())
# print("Sample Product_Description:", df['Product_Description'].head().tolist())
# print("Sample Comment_Text:", comments_df['Comment_Text'].head().tolist())
# print("Comments_df Campaign_ID matches:", comments_df['Campaign_ID'].isin(df['Campaign_ID']).sum())

# Ensure Comment_Text is string and handle non-string values
print("Converting Comment_Text to strings...")
comments_df = comments_df.copy()
comments_df['Comment_Text'] = comments_df['Comment_Text'].apply(lambda x: str(x) if not pd.isna(x) else '')
comments_df['Comment_Text'] = comments_df['Comment_Text'].replace('nan', '')

# Recompute comment sentiment and aggregates
print("Computing comment sentiment...")
comments_df['Comment_Sentiment'] = comments_df['Comment_Text'].apply(lambda x: TextBlob(x).sentiment.polarity if x.strip() else 0.0)
comment_agg = comments_df.groupby('Campaign_ID').agg({
    'Comment_Sentiment': 'mean',
    'Comment_Text': ['count', lambda x: ' '.join(x) if x.str.strip().any() else ''],
    'Likes': 'sum'
}).reset_index()
comment_agg.columns = ['Campaign_ID', 'Avg_Comment_Sentiment', 'Comment_Count', 'Comments', 'Likes_sum']

# Drop conflicting columns from df before merge
conflicting_columns = ['Avg_Comment_Sentiment', 'Comment_Count', 'Comments', 'Likes_sum']
existing_conflicts = [col for col in conflicting_columns if col in df.columns]
if existing_conflicts:
    print(f"Dropping conflicting columns from df: {existing_conflicts}")
    df = df.drop(columns=existing_conflicts)

# Merge with ad dataframe
print("Merging df with comment_agg...")
df = df.merge(comment_agg, on='Campaign_ID', how='left')
df['Avg_Comment_Sentiment'] = df['Avg_Comment_Sentiment'].fillna(0.0)
df['Comment_Count'] = df['Comment_Count'].fillna(0)
df['Comments'] = df['Comments'].fillna('')
df['Likes_sum'] = df['Likes_sum'].fillna(0)

# Verify merged data
print("Feature statistics after merge:")
print(df[['Sentiment_Score', 'Avg_Comment_Sentiment', 'Comment_Count', 'Likes_sum']].describe())

# Handle NaN in text
df['Ad_Text'] = df['Ad_Text'].fillna('')
df['Product_Description'] = df['Product_Description'].fillna('')
comments_df['Comment_Text'] = comments_df['Comment_Text'].fillna('')

# Drop existing TF-IDF columns
tfidf_columns = [col for col in df.columns if col.startswith('comment_tfidf_')]
if tfidf_columns:
    print(f"Dropping existing TF-IDF columns from df: {tfidf_columns}")
    df = df.drop(columns=tfidf_columns)

# Text preprocessing
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    text = str(text)
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    words = [lemmatizer.lemmatize(word) for word in text.split() if len(word) > 2 and word not in {'http', 'www', 'com'}]
    return ' '.join(words) if words else 'neutral'

df['Ad_Text'] = df['Ad_Text'].apply(preprocess_text)
df['Product_Description'] = df['Product_Description'].apply(preprocess_text)
comments_df['Comment_Text'] = comments_df['Comment_Text'].apply(preprocess_text)

# Concatenate comments for TF-IDF
comments_concat = comments_df.groupby('Campaign_ID')['Comment_Text'].apply(lambda x: ' '.join(x)).reset_index()

# Extract TF-IDF features for combined text
tfidf = TfidfVectorizer(max_features=100, stop_words='english')
all_text = df['Ad_Text'] + ' ' + df['Product_Description'] + ' ' + comments_concat['Comment_Text'].reindex(df.index, fill_value='neutral')
text_tfidf = tfidf.fit_transform(all_text)

# Verify TF-IDF features
print("TF-IDF feature sparsity:", (text_tfidf.nnz / (text_tfidf.shape[0] * text_tfidf.shape[1])) * 100, "% non-zero")

# One-hot encode categorical features
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
categorical_features = encoder.fit_transform(df[['Platform', 'Category', 'Sentiment_Label']])
categorical_feature_names = encoder.get_feature_names_out(['Platform', 'Category', 'Sentiment_Label'])

# Combine numerical features with scaling
numerical_features = df[['Sentiment_Score', 'Avg_Comment_Sentiment', 'Comment_Count', 'Likes_sum']]
numerical_feature_names = ['Sentiment_Score', 'Avg_Comment_Sentiment', 'Comment_Count', 'Likes_sum']
scaler = StandardScaler()
numerical_features_scaled = scaler.fit_transform(numerical_features)

# Compute means for numerical features (for inference)
numerical_means = numerical_features.mean()

# Combine all features for training
X = hstack([text_tfidf, numerical_features_scaled, categorical_features])
y = df['CTR'].values

# Feature names for importance
feature_names = (
    [f'text_tfidf_{i}' for i in range(text_tfidf.shape[1])] +
    numerical_feature_names +
    list(categorical_feature_names)
)

# Verify feature alignment
print(f"X shape: {X.shape}, Expected features: {text_tfidf.shape[1] + numerical_features_scaled.shape[1] + categorical_features.shape[1]}")

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Model Performance (Training with All Features):")
print(f"Mean Squared Error: {mse:.6f}")
print(f"Mean Absolute Error: {mae:.6f}")
print(f"R² Score: {r2:.4f}")

# Feature importance
if len(feature_names) != X.shape[1]:
    print(f"Warning: Feature names length ({len(feature_names)}) does not match X columns ({X.shape[1]})")
    feature_names = feature_names[:X.shape[1]]
    if len(feature_names) < X.shape[1]:
        feature_names.extend([f'unknown_feature_{i}' for i in range(X.shape[1] - len(feature_names))])

importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False).head(10)

print("\nTop 10 Most Important Features:")
print(feature_importance_df)

# Visualize feature importance
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
plt.title('Top 10 Feature Importances for CTR Prediction')
plt.tight_layout()
plt.savefig('/kaggle/working/feature_importance.png')
plt.close()


Estimating campaign success based on slogan, description, likes, and comments...
Converting Comment_Text to strings...
Computing comment sentiment...
Dropping conflicting columns from df: ['Avg_Comment_Sentiment', 'Comment_Count', 'Comments']
Merging df with comment_agg...
Feature statistics after merge:
       Sentiment_Score  Avg_Comment_Sentiment  Comment_Count    Likes_sum
count      2000.000000            2000.000000         2000.0  2000.000000
mean          0.201077               0.180871           10.0   162.275500
std           0.105345               0.098084            0.0    60.721933
min          -0.210000              -0.145000           10.0    46.000000
25%           0.131234               0.114736           10.0   104.000000
50%           0.193781               0.176694           10.0   169.000000
75%           0.264307               0.245671           10.0   210.000000
max           0.800000               0.540833           10.0   325.000000
TF-IDF feature sparsity: 46.

In [12]:
df

,Campaign_ID,Ad_Text,Product_Description,Category,Platform,Date,Sentiment_Score,Sentiment_Label,Impressions,Money_Spent,...,Conversions,Conversion_Rate,Cost_Per_Conversion,Engagement_Rate,Brand_Sentiment_Lift,Share_of_Voice,Avg_Comment_Sentiment,Comment_Count,Comments,Likes_sum
0,C000,breakfast healthy event for all everyones favo...,click the image below launch content calendar ...,Food & Beverage,Facebook,2025-01-01 00:00:00.000000000,0.233333,Positive,911321,65286,...,1142,0.001253,57.168126,0.000550,0.0,0.003642,0.214586,10,@user I think I may have to start watching the...,190
1,C001,make difference the product description for th...,learn drive sale for health and wellness welln...,Beauty,Instagram,2025-01-01 04:22:12.666333166,0.092929,Neutral,454865,75842,...,357,0.000785,212.442577,0.000141,0.0,0.001847,0.200564,10,You may be surprised to know that my last twee...,197
2,C002,dont judge product it product description take...,make email marketing campaign targeting young ...,Apparel,Facebook,2025-01-01 08:44:25.332666333,0.326326,Positive,437826,15908,...,634,0.001448,25.091483,0.000697,0.0,0.001551,0.373902,10,I am in my 30s and I am addicted to watching S...,203
3,C003,biscuitfree it called freebie contest the cont...,use their social medium influencer strategy pr...,Food & Beverage,Facebook,2025-01-01 13:06:37.998999499,0.108838,Positive,675418,93617,...,1185,0.001754,79.001688,0.000490,0.0,0.002699,0.122625,10,I may be a bit disappointed in the amount of w...,128
4,C004,please support this awesome ecofriendly produc...,click here read about sustainable food deliver...,Food & Beverage,Twitter,2025-01-01 17:28:50.665332666,0.098674,Neutral,265948,51626,...,137,0.000515,376.832117,0.000466,0.0,0.001063,0.200013,10,@user @user @user I\u2019m going to go to the ...,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,C1995,make difference with fashion start fashion sho...,promote new ecofriendly fashion brand launch n...,Apparel,Facebook,2025-12-30 06:31:09.334667336,0.090909,Neutral,831935,57597,...,767,0.000922,75.093872,0.000281,0.0,0.002947,0.274352,10,The fact that the Knicks are now playing the N...,181
1996,C1996,think big and act influencer for new ecofriend...,introducing aipowered social medium campaign p...,Electronics,Instagram,2025-12-30 10:53:22.001000500,0.043669,Neutral,496675,46212,...,347,0.000699,133.175793,0.000560,0.0,0.001998,0.101111,10,I can't believe I'm going to see the Foo Fight...,195
1997,C1997,inspire lifestyle brand instagram page for sof...,introducing ecofriendly ecofriendly fashion br...,Apparel,Facebook,2025-12-30 15:15:34.667333668,0.239322,Positive,85403,10637,...,36,0.000422,295.472222,0.000281,0.0,0.000302,0.025963,10,If you want to make the trip to the Foo Fighte...,255
1998,C1998,this isnt about animal welfare this campaign i...,how can promote new food program for young chi...,Food & Beverage,Twitter,2025-12-30 19:37:47.333666836,0.092045,Neutral,968762,44258,...,703,0.000726,62.955903,0.000154,0.0,0.003872,0.162220,10,The best part about my new Apple Watch is that...,97


> Objective: Predict CTR (Click-Through Rate, range 0.005–0.03) for 2000 ad campaigns using features derived from:
Slogan (Ad_Text): TF-IDF features (up to 500) capturing key terms.
Description (Product_Description): TF-IDF features (up to 500).
Likes (Likes_sum): Sum of Likes from ad_comments.csv per campaign.
Comments:
Comment_Text TF-IDF (up to 500 features) from concatenated comments.
Avg_Comment_Sentiment: Mean sentiment score of comments.
Comment_Count: Number of comments per campaign.
Categorical Features: One-hot encoded Platform (Twitter, Instagram, Facebook), Category (Apparel, Electronics, Food & Beverage, Beauty), Sentiment_Label (Positive, Negative, Neutral).
Process:
Loaded intermediate_ads.csv (ad campaigns) and ad_comments.csv (comments).
Aggregated Likes_sum from ad_comments.csv.
Generated TF-IDF features for text columns using separate TfidfVectorizer instances (max_features=500).
Merged Comment_Text TF-IDF features into df, handling NaNs and dropping duplicate columns.
Combined features into a matrix X (~1513 columns, depending on TF-IDF sizes).
Split data (80% train, 20% test), trained a Random Forest Regressor (n_estimators=100), and predicted CTR.
Evaluated with MSE, MAE, R²; saved predictions and feature importances.

In [13]:

# --- Inference Pipeline: Predict CTR using only Ad_Text and Product_Description ---
def predict_ctr_batch(ad_texts, product_descriptions, tfidf_vectorizer, encoder, scaler, numerical_means, categorical_default, model):
    # Prepare text inputs
    text_inputs = pd.Series([f"{str(ad)} {str(desc)} neutral" for ad, desc in zip(ad_texts, product_descriptions)])
    text_tfidf_inf = tfidf_vectorizer.transform(text_inputs)
    
    # Numerical features: Use mean values for all rows
    numerical_inf = pd.DataFrame([numerical_means] * len(ad_texts), columns=numerical_feature_names)
    numerical_inf_scaled = scaler.transform(numerical_inf)
    
    # Categorical features: Use default values for all rows
    categorical_df = pd.DataFrame([categorical_default] * len(ad_texts), columns=['Platform', 'Category', 'Sentiment_Label'])
    categorical_inf = encoder.transform(categorical_df)
    
    # Combine features
    X_inf = hstack([text_tfidf_inf, numerical_inf_scaled, categorical_inf])
    
    # Predict
    return model.predict(X_inf)

# Define default categorical values (use most frequent from training set)
categorical_default = {
    'Platform': df['Platform'].mode()[0],
    'Category': df['Category'].mode()[0],
    'Sentiment_Label': df['Sentiment_Label'].mode()[0]
}

# Predict CTR for all campaigns using only Ad_Text and Product_Description
df['Predicted_CTR'] = predict_ctr_batch(
    df['Ad_Text'], 
    df['Product_Description'], 
    tfidf, 
    encoder, 
    scaler, 
    numerical_means, 
    categorical_default, 
    rf_model
)

# Save predictions to CSV
df[['Campaign_ID', 'Ad_Text', 'Product_Description', 'CTR', 'Predicted_CTR', 'Sentiment_Score', 'Avg_Comment_Sentiment', 'Likes_sum']].to_csv(
    '/kaggle/working/campaign_success.csv', 
    index=False
)
print(f"Campaign success predictions saved to /kaggle/working/campaign_success.csv")

# Display sample predictions
print("\nSample Campaign Success Predictions:")
print(df[['Campaign_ID', 'Ad_Text', 'Product_Description', 'CTR', 'Predicted_CTR', 'Sentiment_Score', 'Avg_Comment_Sentiment']].head())

# Debug feature statistics
print("\nNumerical feature means (used for inference):")
print(numerical_means)
print("\nDefault categorical values (used for inference):")
print(categorical_default)

Campaign success predictions saved to /kaggle/working/campaign_success.csv

Sample Campaign Success Predictions:
  Campaign_ID                                            Ad_Text  \
0        C000  breakfast healthy event for all everyones favo...   
1        C001  make difference the product description for th...   
2        C002  dont judge product it product description take...   
3        C003  biscuitfree it called freebie contest the cont...   
4        C004  please support this awesome ecofriendly produc...   

                                 Product_Description       CTR  Predicted_CTR  \
0  click the image below launch content calendar ...  0.012484       0.010855   
1  learn drive sale for health and wellness welln...  0.009309       0.011672   
2  make email marketing campaign targeting young ...  0.013238       0.010743   
3  use their social medium influencer strategy pr...  0.017615       0.010727   
4  click here read about sustainable food deliver...  0.008829       0.01

XGBOOST RESULTS

In [14]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.sparse import hstack
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor

# --- Data Preparation ---
# Ensure no duplicate columns
df = df.loc[:, ~df.columns.duplicated()]
print("Original columns in df:", df.columns.tolist())
print("Duplicate columns in df:", df.columns[df.columns.duplicated()].tolist())

# Define target
y = df['CTR'].values  # Assuming CTR is the target; replace with 'Conversions' if needed

# --- Feature Engineering for Training ---
# Text features
all_text = df['Ad_Text'] + ' ' + df['Product_Description'] + ' ' + comments_concat['Comment_Text'].reindex(df.index, fill_value='')
tfidf = TfidfVectorizer(max_features=500, stop_words='english')
text_tfidf = tfidf.fit_transform(all_text)

# Numerical features
numerical_features = df[['Avg_Comment_Sentiment', 'Comment_Count', 'Likes_sum', 'Sentiment_Score']].values
numerical_feature_names = ['Avg_Comment_Sentiment', 'Comment_Count', 'Likes_sum', 'Sentiment_Score']

# Compute means for numerical features (for inference)
numerical_means = df[['Avg_Comment_Sentiment', 'Comment_Count', 'Likes_sum', 'Sentiment_Score']].mean().values

# Categorical features
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
categorical_features = encoder.fit_transform(df[['Platform', 'Category', 'Sentiment_Label']])
categorical_feature_names = encoder.get_feature_names_out(['Platform', 'Category', 'Sentiment_Label'])

# Combine features for training
X = hstack([text_tfidf, numerical_features, categorical_features])

# Feature names for importance
feature_names = (
    [f'text_tfidf_{i}' for i in range(text_tfidf.shape[1])] +
    numerical_feature_names +
    list(categorical_feature_names)
)

# Verify feature alignment
print(f"X shape: {X.shape}, Expected features: {text_tfidf.shape[1] + numerical_features.shape[1] + categorical_features.shape[1]}")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Train XGBoost Model ---
model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# --- Evaluate on Test Set ---
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Model Performance (Training with All Features):")
print(f"Mean Squared Error: {mse:.6f}")
print(f"Mean Absolute Error: {mae:.6f}")
print(f"R² Score: {r2:.4f}")

# Feature importance
importances = model.feature_importances_
if len(feature_names) != len(importances):
    print(f"Error: Feature names length ({len(feature_names)}) does not match importances length ({len(importances)})")
    feature_names = feature_names[:len(importances)]  # Truncate if needed
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False).head(10)

print("\nTop 10 Most Important Features:")
print(feature_importance_df)

# Visualize feature importance
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
plt.title('Top 10 Feature Importances for CTR Prediction (XGBoost)')
plt.tight_layout()
plt.savefig('/kaggle/working/xgboost_feature_importance.png')
plt.close()

# --- Inference Pipeline: Predict CTR using only Ad_Text and Product_Description ---
def predict_ctr(ad_text, product_description, tfidf_vectorizer, encoder, numerical_means, categorical_default, model):
    # Prepare text input
    text_input = pd.Series([str(ad_text) + ' ' + str(product_description)])
    text_tfidf_inf = tfidf_vectorizer.transform(text_input)
    
    # Numerical features: Use mean values
    numerical_inf = np.array([numerical_means])
    
    # Categorical features: Use default values
    categorical_df = pd.DataFrame([categorical_default], columns=['Platform', 'Category', 'Sentiment_Label'])
    categorical_inf = encoder.transform(categorical_df)
    
    # Combine features
    X_inf = hstack([text_tfidf_inf, numerical_inf, categorical_inf])
    
    # Predict
    return model.predict(X_inf)[0]

# Define default categorical values (use most frequent from training set)
categorical_default = {
    'Platform': df['Platform'].mode()[0],
    'Category': df['Category'].mode()[0],
    'Sentiment_Label': df['Sentiment_Label'].mode()[0]
}

# Predict CTR for all campaigns using only Ad_Text and Product_Description
df['Predicted_CTR'] = df.apply(
    lambda row: predict_ctr(
        row['Ad_Text'], 
        row['Product_Description'], 
        tfidf, 
        encoder, 
        numerical_means, 
        categorical_default, 
        model
    ), 
    axis=1
)

# Save predictions to CSV
df[['Campaign_ID', 'Ad_Text', 'Product_Description', 'CTR', 'Predicted_CTR', 'Avg_Comment_Sentiment', 'Likes_sum']].to_csv(
    '/kaggle/working/xgboost_campaign_success.csv', 
    index=False
)
print(f"Campaign success predictions saved to /kaggle/working/xgboost_campaign_success.csv")

# Display sample predictions
print("\nSample Campaign Success Predictions:")
print(df[['Campaign_ID', 'Ad_Text', 'Product_Description', 'CTR', 'Predicted_CTR']].head())

# Debug feature statistics
print("\nNumerical feature means (used for inference):")
print(pd.Series(numerical_means, index=numerical_feature_names))
print("\nDefault categorical values (used for inference):")
print(categorical_default)

Original columns in df: ['Campaign_ID', 'Ad_Text', 'Product_Description', 'Category', 'Platform', 'Date', 'Sentiment_Score', 'Sentiment_Label', 'Impressions', 'Money_Spent', 'CTR', 'ROI', 'Goods_Sold', 'Likes', 'Retweets', 'Conversions', 'Conversion_Rate', 'Cost_Per_Conversion', 'Engagement_Rate', 'Brand_Sentiment_Lift', 'Share_of_Voice', 'Avg_Comment_Sentiment', 'Comment_Count', 'Comments', 'Likes_sum', 'Predicted_CTR']
Duplicate columns in df: []
X shape: (2000, 514), Expected features: 514
XGBoost Model Performance (Training with All Features):
Mean Squared Error: 0.000021
Mean Absolute Error: 0.003655
R² Score: -0.0545

Top 10 Most Important Features:
            Feature  Importance
306  text_tfidf_306    0.012111
394  text_tfidf_394    0.009264
449  text_tfidf_449    0.007064
271  text_tfidf_271    0.006754
148  text_tfidf_148    0.006706
57    text_tfidf_57    0.006702
145  text_tfidf_145    0.006364
379  text_tfidf_379    0.006256
472  text_tfidf_472    0.006167
190  text_tfidf_

Trying other things

In [15]:
df['CTR'] = [
    min(0.1, max(0.001, 
        0.05 * s +  # Sentiment_Score contribution
        0.02 * (l / df['Impressions'].mean()) +  # Likes_sum contribution
        0.03 * a +  # Avg_Comment_Sentiment contribution
        np.random.normal(0.02, 0.01)
    )) 
    for s, l, a in zip(df['Sentiment_Score'], df['Likes_sum'], df['Avg_Comment_Sentiment'])
]

In [16]:
df['Conversions'] = [int(i * c * np.random.uniform(0.05, 0.15)) for i, c in zip(df['Impressions'], df['CTR'])]

In [17]:
# --- Campaign Success Prediction ---
print("Estimating campaign success based on Conversions...")

# Create a clean DataFrame with essential columns
essential_columns = [
    'Campaign_ID', 'Ad_Text', 'Product_Description', 'Category', 'Platform', 
    'Sentiment_Score', 'Sentiment_Label', 'Impressions', 'Conversions'
]
df_clean = df[essential_columns].copy()

# Check for duplicate columns in original df
print("Original columns in df:", df.columns.tolist())
print("Duplicate columns in df:", df.columns[df.columns.duplicated()].tolist())

# Recompute Avg_Comment_Sentiment and Comment_Count from comments_df
comments_df['Comment_Sentiment'] = comments_df['Comment_Text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
comment_agg = comments_df.groupby('Campaign_ID').agg({
    'Comment_Sentiment': 'mean',
    'Comment_Text': 'count',
    'Likes': 'sum'
}).reset_index().rename(columns={
    'Comment_Sentiment': 'Avg_Comment_Sentiment',
    'Comment_Text': 'Comment_Count',
    'Likes': 'Likes_sum'
})

# Merge with clean DataFrame
df_clean = df_clean.merge(comment_agg, on='Campaign_ID', how='left')
df_clean['Avg_Comment_Sentiment'] = df_clean['Avg_Comment_Sentiment'].fillna(0)
df_clean['Comment_Count'] = df_clean['Comment_Count'].fillna(0)
df_clean['Likes_sum'] = df_clean['Likes_sum'].fillna(0)

# Verify columns and data
print("Columns after merge:", df_clean.columns.tolist())
print("Duplicate columns after merge:", df_clean.columns[df_clean.columns.duplicated()].tolist())
print("Feature statistics:")
print(df_clean[['Sentiment_Score', 'Avg_Comment_Sentiment', 'Comment_Count', 'Likes_sum']].describe())

# Handle NaN in text
df_clean['Ad_Text'] = df_clean['Ad_Text'].fillna('')
df_clean['Product_Description'] = df_clean['Product_Description'].fillna('')
comments_df['Comment_Text'] = comments_df['Comment_Text'].fillna('')

# Text preprocessing
from nltk.stem import WordNetLemmatizer
import re
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    words = [lemmatizer.lemmatize(word) for word in text.split() if len(word) > 2]
    return ' '.join(words)
df_clean['Ad_Text'] = df_clean['Ad_Text'].apply(preprocess_text)
df_clean['Product_Description'] = df_clean['Product_Description'].apply(preprocess_text)
comments_df['Comment_Text'] = comments_df['Comment_Text'].apply(preprocess_text)

# Filter low-content comments
comments_df = comments_df[comments_df['Comment_Text'].str.len() >= 5]
comments_concat = comments_df.groupby('Campaign_ID')['Comment_Text'].apply(lambda x: ' '.join(x)).reset_index()

# Add interaction features
df_clean['Sentiment_Likes_Interaction'] = df_clean['Sentiment_Score'] * df_clean['Likes_sum']
df_clean['Comment_Sentiment_Count'] = df_clean['Avg_Comment_Sentiment'] * df_clean['Comment_Count']

# Verify interaction features
print("Interaction features created successfully:")
print(df_clean[['Sentiment_Likes_Interaction', 'Comment_Sentiment_Count']].describe())

# Extract TF-IDF features
all_text = df_clean['Ad_Text'] + ' ' + df_clean['Product_Description'] + ' ' + comments_concat['Comment_Text'].reindex(df_clean.index, fill_value='')
tfidf = TfidfVectorizer(max_features=200, stop_words='english')
text_tfidf = tfidf.fit_transform(all_text)

# One-hot encode categorical features
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
categorical_features = encoder.fit_transform(df_clean[['Platform', 'Category', 'Sentiment_Label']])
categorical_feature_names = encoder.get_feature_names_out(['Platform', 'Category', 'Sentiment_Label'])

# Scale numerical features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numerical_features = scaler.fit_transform(df_clean[[
    'Avg_Comment_Sentiment', 'Comment_Count', 'Likes_sum', 'Sentiment_Score',
    'Sentiment_Likes_Interaction', 'Comment_Sentiment_Count'
]].values)

# Combine features
from scipy.sparse import hstack
X = hstack([text_tfidf, numerical_features, categorical_features])
y = df_clean['Conversions'].values  # Predict Conversions

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Simplified XGBoost with early stopping
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
model = XGBRegressor(
    n_estimators=100, learning_rate=0.1, max_depth=5, reg_lambda=1, 
    random_state=42, n_jobs=2
)
model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='rmse',
    early_stopping_rounds=10,
    verbose=False
)

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Model Performance (Conversions):")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Mean Absolute Error: {mae:.2f}")
print(f"R² Score: {r2:.4f}")

# Cross-validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=5, scoring='r2', n_jobs=2)
print(f"Cross-Validated R²: {scores.mean():.4f} ± {scores.std():.4f}")

# Feature importance
feature_names = (
    [f'text_tfidf_{i}' for i in range(text_tfidf.shape[1])] +
    ['Avg_Comment_Sentiment', 'Comment_Count', 'Likes_sum', 'Sentiment_Score',
     'Sentiment_Likes_Interaction', 'Comment_Sentiment_Count'] +
    list(categorical_feature_names)
)
importances = model.feature_importances_
if len(feature_names) != len(importances):
    print(f"Warning: Feature names length ({len(feature_names)}) does not match importances ({len(importances)})")
    feature_names = feature_names[:len(importances)]
    if len(feature_names) < len(importances):
        feature_names.extend([f'unknown_feature_{i}' for i in range(len(importances) - len(feature_names))])

feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False).head(10)

print("\nTop 10 Most Important Features:")
print(feature_importance_df)

# Visualize feature importance
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
plt.title('Top 10 Feature Importances for Conversions Prediction')
plt.tight_layout()
plt.savefig('/kaggle/working/xgboost_conversions_feature_importance.png')
plt.close()

# Predict for all campaigns
df_clean = pd.concat([df_clean, pd.Series(model.predict(X), index=df_clean.index, name='Predicted_Conversions')], axis=1)
df_clean[['Campaign_ID', 'Ad_Text', 'Product_Description', 'Conversions', 'Predicted_Conversions', 
          'Avg_Comment_Sentiment', 'Likes_sum']].to_csv('/kaggle/working/xgboost_conversions_success.csv', index=False)
print(f"Campaign success predictions saved to /kaggle/working/xgboost_conversions_success.csv")

# Display sample predictions
print("\nSample Campaign Success Predictions:")
print(df_clean[['Campaign_ID', 'Ad_Text', 'Conversions', 'Predicted_Conversions']].head())

Estimating campaign success based on Conversions...
Original columns in df: ['Campaign_ID', 'Ad_Text', 'Product_Description', 'Category', 'Platform', 'Date', 'Sentiment_Score', 'Sentiment_Label', 'Impressions', 'Money_Spent', 'CTR', 'ROI', 'Goods_Sold', 'Likes', 'Retweets', 'Conversions', 'Conversion_Rate', 'Cost_Per_Conversion', 'Engagement_Rate', 'Brand_Sentiment_Lift', 'Share_of_Voice', 'Avg_Comment_Sentiment', 'Comment_Count', 'Comments', 'Likes_sum', 'Predicted_CTR']
Duplicate columns in df: []
Columns after merge: ['Campaign_ID', 'Ad_Text', 'Product_Description', 'Category', 'Platform', 'Sentiment_Score', 'Sentiment_Label', 'Impressions', 'Conversions', 'Avg_Comment_Sentiment', 'Comment_Count', 'Likes_sum']
Duplicate columns after merge: []
Feature statistics:
       Sentiment_Score  Avg_Comment_Sentiment  Comment_Count    Likes_sum
count      2000.000000            2000.000000         2000.0  2000.000000
mean          0.201077               0.167481           10.0   162.275500
s

/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBoost Model Performance (Conversions):
Mean Squared Error: 1838987.73
Mean Absolute Error: 1057.86
R² Score: -0.0082
Cross-Validated R²: -0.0555 ± 0.0397

Top 10 Most Important Features:
                   Feature  Importance
203        Sentiment_Score    0.018752
101         text_tfidf_101    0.017024
153         text_tfidf_153    0.016055
57           text_tfidf_57    0.015289
107         text_tfidf_107    0.014708
62           text_tfidf_62    0.014665
85           text_tfidf_85    0.014628
65           text_tfidf_65    0.014217
200  Avg_Comment_Sentiment    0.014086
38           text_tfidf_38    0.014075
Campaign success predictions saved to /kaggle/working/xgboost_conversions_success.csv

Sample Campaign Success Predictions:
  Campaign_ID                                            Ad_Text  Conversions  \
0        C000  breakfast healthy event for all everyones favo...         1457   
1        C001  make difference the product description for th...         1766   
2        C002  

In [18]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import Ridge
from scipy.sparse import hstack
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
import re
import lightgbm as lgb
import catboost as cb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from transformers import pipeline
import optuna
import shap
from textstat import flesch_reading_ease
from datasets import Dataset

# Initialize VADER and RoBERTa (replacing DistilBERT for balanced sentiment)
sid = SentimentIntensityAnalyzer()
sentiment_analyzer = pipeline("sentiment-analysis", model="roberta-base", device=0)  # Use GPU

# --- Campaign Success Prediction ---
print("Estimating campaign success based on Conversions with Stacked Ensemble...")

# Remove duplicate columns from df
df = df.loc[:, ~df.columns.duplicated()]
print("Original columns in df:", df.columns.tolist())
print("Duplicate columns in df:", df.columns[df.columns.duplicated()].tolist())

# Create a clean DataFrame with essential columns
essential_columns = [
    'Campaign_ID', 'Ad_Text', 'Product_Description', 'Category', 'Platform',
    'Sentiment_Score', 'Sentiment_Label', 'Impressions', 'Money_Spent', 'Conversions'
]
df_clean = df[essential_columns].copy()

# Simulate variable Comment_Count (temporary until comment generation code is shared)
np.random.seed(42)
comment_counts = np.random.randint(5, 16, size=len(df_clean))
comment_agg_temp = pd.DataFrame({
    'Campaign_ID': df_clean['Campaign_ID'],
    'Comment_Count': comment_counts
})

# Advanced sentiment analysis with VADER and RoBERTa (batched)
print("Computing advanced sentiment features...")
def batch_sentiment(texts):
    dataset = Dataset.from_dict({"text": [str(t) if str(t).strip() else "neutral" for t in texts]})
    results = sentiment_analyzer(dataset["text"], batch_size=32, truncation=True)
    return [r['score'] if r['label'] == 'POSITIVE' else 1 - r['score'] for r in results]

comments_df['Comment_Sentiment'] = comments_df['Comment_Text'].apply(
    lambda x: sid.polarity_scores(str(x))['compound'] if str(x).strip() else 0.0
)
comments_df['Comment_RoBERTa'] = batch_sentiment(comments_df['Comment_Text'].tolist())
df_clean['Ad_RoBERTa'] = batch_sentiment(df_clean['Ad_Text'].tolist())
df_clean['Desc_RoBERTa'] = batch_sentiment(df_clean['Product_Description'].tolist())

# Aggregate comment features
comment_agg = comments_df.groupby('Campaign_ID').agg({
    'Comment_Sentiment': 'mean',
    'Comment_RoBERTa': 'mean',
    'Comment_Text': 'count',
    'Likes': 'sum'
}).reset_index().rename(columns={
    'Comment_Sentiment': 'Avg_Comment_Sentiment',
    'Comment_RoBERTa': 'Avg_Comment_RoBERTa',
    'Comment_Text': 'Comment_Count_Agg',
    'Likes': 'Likes_sum'
})

# Merge simulated Comment_Count and aggregated features
comment_agg = comment_agg.merge(comment_agg_temp, on='Campaign_ID', how='left')
comment_agg['Comment_Count'] = comment_agg['Comment_Count_Agg'].combine_first(comment_agg['Comment_Count']).clip(5, 15)

# Merge with clean DataFrame
df_clean = df_clean.merge(comment_agg[['Campaign_ID', 'Avg_Comment_Sentiment', 'Avg_Comment_RoBERTa', 'Comment_Count', 'Likes_sum']], on='Campaign_ID', how='left')
df_clean['Avg_Comment_Sentiment'] = df_clean['Avg_Comment_Sentiment'].fillna(0)
df_clean['Avg_Comment_RoBERTa'] = df_clean['Avg_Comment_RoBERTa'].fillna(0.5)
df_clean['Comment_Count'] = df_clean['Comment_Count'].fillna(5)
df_clean['Likes_sum'] = df_clean['Likes_sum'].fillna(0)

# Feature engineering
df_clean['Log_Impressions'] = np.log1p(df_clean['Impressions'].clip(lower=1))
df_clean['Log_Money_Spent'] = np.log1p(df_clean['Money_Spent'].clip(lower=1))
df_clean['Log_Likes_sum'] = np.log1p(df_clean['Likes_sum'].clip(lower=1))
df_clean['Sentiment_Interaction'] = df_clean['Sentiment_Score'] * df_clean['Avg_Comment_Sentiment']
df_clean['Ad_Word_Count'] = df_clean['Ad_Text'].apply(lambda x: len(str(x).split()))
df_clean['Ad_Readability'] = df_clean['Ad_Text'].apply(lambda x: flesch_reading_ease(str(x)) if str(x).strip() else 0)

# Verify features
print("Feature statistics:")
print(df_clean[[
    'Sentiment_Score', 'Avg_Comment_Sentiment', 'Avg_Comment_RoBERTa',
    'Comment_Count', 'Likes_sum', 'Log_Impressions', 'Log_Money_Spent',
    'Sentiment_Interaction', 'Ad_Word_Count', 'Ad_Readability'
]].describe())

# Handle NaN in text
df_clean['Ad_Text'] = df_clean['Ad_Text'].fillna('')
df_clean['Product_Description'] = df_clean['Product_Description'].fillna('')
comments_df['Comment_Text'] = comments_df['Comment_Text'].fillna('')

# Text preprocessing
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    words = [lemmatizer.lemmatize(word) for word in text.split() if len(word) > 2 and word not in {'http', 'www', 'com'}]
    return ' '.join(words) if words else 'neutral'

df_clean['Ad_Text'] = df_clean['Ad_Text'].apply(preprocess_text)
df_clean['Product_Description'] = df_clean['Product_Description'].apply(preprocess_text)
comments_df['Comment_Text'] = comments_df['Comment_Text'].apply(preprocess_text)

# Filter low-content comments
comments_df = comments_df[comments_df['Comment_Text'].str.len() >= 5]
comments_concat = comments_df.groupby('Campaign_ID')['Comment_Text'].apply(lambda x: ' '.join(x)).reset_index()

# Extract TF-IDF features
all_text = df_clean['Ad_Text'] + ' ' + df_clean['Product_Description'] + ' ' + comments_concat['Comment_Text'].reindex(df_clean.index, fill_value='neutral')
tfidf = TfidfVectorizer(max_features=100, stop_words='english', ngram_range=(1, 2))
text_tfidf = tfidf.fit_transform(all_text)

# Apply Truncated SVD
svd = TruncatedSVD(n_components=50, random_state=42)
text_tfidf_reduced = svd.fit_transform(text_tfidf)
print("TF-IDF feature variance ratio:", sum(svd.explained_variance_ratio_))

# One-hot encode categorical features
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
categorical_features = encoder.fit_transform(df_clean[['Platform', 'Category', 'Sentiment_Label']])
categorical_feature_names = encoder.get_feature_names_out(['Platform', 'Category', 'Sentiment_Label'])

# Scale numerical features
scaler = StandardScaler()
numerical_features = scaler.fit_transform(df_clean[[
    'Sentiment_Score', 'Avg_Comment_Sentiment', 'Avg_Comment_RoBERTa',
    'Comment_Count', 'Log_Likes_sum', 'Log_Impressions', 'Log_Money_Spent',
    'Sentiment_Interaction', 'Ad_Word_Count', 'Ad_Readability', 'Ad_RoBERTa', 'Desc_RoBERTa'
]])

# Combine features
X = np.hstack([text_tfidf_reduced, numerical_features, categorical_features])
y = np.log1p(df_clean['Conversions'].values)  # Log-transform Conversions

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Simplified Optuna tuning
def objective_lgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 7),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 10),
        'random_state': 42,
        'force_col_wise': True
    }
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)
    return -cross_val_score(model, X_train, y_train, cv=3, scoring='r2').mean()

def objective_cb(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'depth': trial.suggest_int('depth', 3, 7),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'random_seed': 42
    }
    model = cb.CatBoostRegressor(**params, verbose=0)
    model.fit(X_train, y_train)
    return -cross_val_score(model, X_train, y_train, cv=3, scoring='r2').mean()

# Optimize models
study_lgb = optuna.create_study(direction='minimize')
study_lgb.optimize(objective_lgb, n_trials=10)
lgb_model = lgb.LGBMRegressor(**study_lgb.best_params)

study_cb = optuna.create_study(direction='minimize')
study_cb.optimize(objective_cb, n_trials=10)
cb_model = cb.CatBoostRegressor(**study_cb.best_params, verbose=0)

# Simplified Neural Network
nn_model = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])
nn_model.compile(optimizer='adam', loss='mse')
nn_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0, validation_split=0.2)

# Train base models and get predictions
lgb_model.fit(X_train, y_train)
cb_model.fit(X_train, y_train)
lgb_pred_train = lgb_model.predict(X_train)
lgb_pred_test = lgb_model.predict(X_test)
cb_pred_train = cb_model.predict(X_train)
cb_pred_test = cb_model.predict(X_test)
nn_pred_train = nn_model.predict(X_train, verbose=0).flatten()
nn_pred_test = nn_model.predict(X_test, verbose=0).flatten()

# Stack predictions
stacked_train = np.column_stack((lgb_pred_train, cb_pred_train, nn_pred_train))
stacked_test = np.column_stack((lgb_pred_test, cb_pred_test, nn_pred_test))

# Meta-learner
meta_model = Ridge(alpha=1.0)
meta_model.fit(stacked_train, y_train)
y_pred = meta_model.predict(stacked_test)

# Evaluate (back-transform for metrics)
y_pred_back = np.expm1(y_pred)
y_test_back = np.expm1(y_test)
mse = mean_squared_error(y_test_back, y_pred_back)
mae = mean_absolute_error(y_test_back, y_pred_back)
r2 = r2_score(y_test_back, y_pred_back)

print(f"Stacked Ensemble Model Performance (Conversions):")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Mean Absolute Error: {mae:.2f}")
print(f"R² Score: {r2:.4f}")

# Cross-validation
stacked_scores = cross_val_score(meta_model, stacked_train, y_train, cv=5, scoring='r2')
print(f"Cross-Validated R²: {stacked_scores.mean():.4f} ± {stacked_scores.std():.4f}")

# Feature importance via SHAP
explainer = shap.TreeExplainer(lgb_model)
shap_values = explainer.shap_values(X_train)
feature_names = (
    [f'text_tfidf_svd_{i}' for i in range(text_tfidf_reduced.shape[1])] +
    ['Sentiment_Score', 'Avg_Comment_Sentiment', 'Avg_Comment_RoBERTa',
     'Comment_Count', 'Log_Likes_sum', 'Log_Impressions', 'Log_Money_Spent',
     'Sentiment_Interaction', 'Ad_Word_Count', 'Ad_Readability', 'Ad_RoBERTa', 'Desc_RoBERTa'] +
    list(categorical_feature_names)
)
shap_importance = np.abs(shap_values).mean(axis=0)
feature_importance_df = pd.DataFrame({'Feature': feature_names[:len(shap_importance)], 'Importance': shap_importance})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False).head(10)

print("\nTop 10 Most Important Features (SHAP):")
print(feature_importance_df)

# Visualize feature importance
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
plt.title('Top 10 Feature Importances for Conversions Prediction (SHAP)')
plt.tight_layout()
plt.savefig('/kaggle/working/stacked_conversions_feature_importance.png')
plt.close()

# Predict for all campaigns
stacked_all = np.column_stack((
    lgb_model.predict(X),
    cb_model.predict(X),
    nn_model.predict(X, verbose=0).flatten()
))
df_clean['Predicted_Conversions'] = np.expm1(meta_model.predict(stacked_all))
df_clean[['Campaign_ID', 'Ad_Text', 'Product_Description', 'Conversions', 'Predicted_Conversions',
          'Sentiment_Score', 'Avg_Comment_Sentiment', 'Avg_Comment_RoBERTa']].to_csv(
    '/kaggle/working/stacked_conversions_success.csv', index=False
)
print(f"Campaign success predictions saved to /kaggle/working/stacked_conversions_success.csv")

# Display sample predictions
print("\nSample Campaign Success Predictions:")
print(df_clean[['Campaign_ID', 'Ad_Text', 'Conversions', 'Predicted_Conversions', 'Sentiment_Score', 'Avg_Comment_Sentiment']].head())

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-05-14 11:36:05,770] Trial 9 finished with value: -0.7425643493978882 and parameters: {'n_estimators': 103, 'learning_rate': 0.02228383593098045, 'max_depth': 3, 'reg_lambda': 7.8540979673524784}. Best is trial 1 with value: -0.7587632307375735.
[I 2025-05-14 11:36:05,771] A new study created in memory with name: no-name-66dbc3ea-f51a-47de-8204-e8390eebc8f4


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-05-14 11:36:09,158] Trial 0 finished with value: -0.761017217642233 and parameters: {'iterations': 184, 'learning_rate': 0.05510429684525251, 'depth': 5, 'l2_leaf_reg': 1.7161538537879393}. Best is trial 0 with value: -0.761017217642233.
[I 2025-05-14 11:36:12,321] Trial 1 finished with value: -0.76571499850691 and parameters: {'iterations': 274, 'learning_rate': 0.04100688732537081, 'depth': 4, 'l2_leaf_reg': 3.9859182374791047}. Best is trial 1 with value: -0.76571499850691.
[I 2025-05-14 11:36:16,849] Trial 2 finished with value: -0.7570420011745665 and parameters: {'iterations': 167, 'learning_rate': 0.07264805257649007, 'depth': 6, 'l2_leaf_reg': 7.112238242298586}. Best is trial 1 with value: -0.76571499850691.
[I 2025-05-14 11:36:27,271] Trial 3 finished with value: -0.7462882961807288 and parameters: {'iterations': 192, 'learning_rate': 0.04052977654125168, 'depth': 7, 'l2_leaf_reg': 6.504725186636136}. Best is trial 1 with value: -0.76571499850691.
[I 2025-05-14 11:36:

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000793 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15315
[LightGBM] [Info] Number of data points in the train set: 1600, number of used features: 71
[LightGBM] [Info] Start training from score 7.171207
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

In [19]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
import re
import lightgbm as lgb
import catboost as cb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline
from datasets import Dataset
import optuna
import shap
from textstat import flesch_reading_ease
import torch

# Initialize VADER
sid = SentimentIntensityAnalyzer()

# --- Fine-Tune DistilBERT ---
print("Preparing data for fine-tuning DistilBERT...")

# Pseudo-label comments using VADER
def label_comment(score):
    if score > 0.2:
        return 1  # Positive
    elif score < -0.2:
        return 0  # Negative
    else:
        return 2  # Neutral

comments_df['Label'] = comments_df['Comment_Sentiment'].apply(label_comment)

# Sample 1000 comments (or all if <1000) for fine-tuning
train_comments = comments_df[['Comment_Text', 'Label']].sample(n=min(1000, len(comments_df)), random_state=42)
train_comments = train_comments[train_comments['Comment_Text'].str.strip() != ''].dropna()

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_comments.rename(columns={'Comment_Text': 'text', 'Label': 'label'}))

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Define training arguments
training_args = TrainingArguments(
    output_dir="/kaggle/working/distilbert-finetuned",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir='/kaggle/working/logs',
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

# Fine-tune model
print("Fine-tuning DistilBERT...")
trainer.train()

# Save fine-tuned model
model.save_pretrained("/kaggle/working/distilbert-finetuned")
tokenizer.save_pretrained("/kaggle/working/distilbert-finetuned")

# Load fine-tuned model for inference
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="/kaggle/working/distilbert-finetuned",
    tokenizer="/kaggle/working/distilbert-finetuned",
    device=0
)

# --- Campaign Success Prediction ---
print("Estimating campaign success based on Conversions with Stacked Ensemble...")

# Remove duplicate columns from df
df = df.loc[:, ~df.columns.duplicated()]
print("Original columns in df:", df.columns.tolist())
print("Duplicate columns in df:", df.columns[df.columns.duplicated()].tolist())

# Create a clean DataFrame
essential_columns = [
    'Campaign_ID', 'Ad_Text', 'Product_Description', 'Category', 'Platform',
    'Sentiment_Score', 'Sentiment_Label', 'Impressions', 'Money_Spent', 'Conversions'
]
df_clean = df[essential_columns].copy()

# Simulate variable Comment_Count
np.random.seed(42)
comment_counts = np.random.randint(5, 16, size=len(df_clean))
comment_agg_temp = pd.DataFrame({
    'Campaign_ID': df_clean['Campaign_ID'],
    'Comment_Count': comment_counts
})

# Advanced sentiment analysis with VADER and fine-tuned DistilBERT
print("Computing advanced sentiment features...")
def batch_sentiment(texts):
    dataset = Dataset.from_dict({"text": [str(t) if str(t).strip() else "neutral" for t in texts]})
    results = sentiment_analyzer(dataset["text"], batch_size=32, truncation=True)
    score_map = {'LABEL_1': 1.0, 'LABEL_0': -1.0, 'LABEL_2': 0.0}
    return [score_map[r['label']] * r['score'] for r in results]

comments_df['Comment_Sentiment'] = comments_df['Comment_Text'].apply(
    lambda x: sid.polarity_scores(str(x))['compound'] if str(x).strip() else 0.0
)
comments_df['Comment_DistilBERT'] = batch_sentiment(comments_df['Comment_Text'].tolist())
df_clean['Ad_DistilBERT'] = batch_sentiment(df_clean['Ad_Text'].tolist())
df_clean['Desc_DistilBERT'] = batch_sentiment(df_clean['Product_Description'].tolist())

# Aggregate comment features
comment_agg = comments_df.groupby('Campaign_ID').agg({
    'Comment_Sentiment': 'mean',
    'Comment_DistilBERT': 'mean',
    'Comment_Text': 'count',
    'Likes': 'sum'
}).reset_index().rename(columns={
    'Comment_Sentiment': 'Avg_Comment_Sentiment',
    'Comment_DistilBERT': 'Avg_Comment_DistilBERT',
    'Comment_Text': 'Comment_Count_Agg',
    'Likes': 'Likes_sum'
})

# Merge and enforce variable Comment_Count
comment_agg = comment_agg.merge(comment_agg_temp, on='Campaign_ID', how='left')
comment_agg['Comment_Count'] = comment_agg['Comment_Count'].clip(5, 15)

# Merge with clean DataFrame
df_clean = df_clean.merge(comment_agg[['Campaign_ID', 'Avg_Comment_Sentiment', 'Avg_Comment_DistilBERT', 'Comment_Count', 'Likes_sum']], on='Campaign_ID', how='left')
df_clean['Avg_Comment_Sentiment'] = df_clean['Avg_Comment_Sentiment'].fillna(0)
df_clean['Avg_Comment_DistilBERT'] = df_clean['Avg_Comment_DistilBERT'].fillna(0)
df_clean['Comment_Count'] = df_clean['Comment_Count'].fillna(5)
df_clean['Likes_sum'] = df_clean['Likes_sum'].fillna(0)

# Feature engineering
df_clean['Log_Impressions'] = np.log1p(df_clean['Impressions'].clip(lower=1, upper=df_clean['Impressions'].quantile(0.95)))
df_clean['Log_Money_Spent'] = np.log1p(df_clean['Money_Spent'].clip(lower=1))
df_clean['Log_Likes_sum'] = np.log1p(df_clean['Likes_sum'].clip(lower=1))
df_clean['Sentiment_Interaction'] = df_clean['Sentiment_Score'] * df_clean['Avg_Comment_Sentiment']
df_clean['Ad_Word_Count'] = df_clean['Ad_Text'].apply(lambda x: len(str(x).split()))
df_clean['Ad_Readability'] = df_clean['Ad_Text'].apply(lambda x: flesch_reading_ease(str(x)) if str(x).strip() else 0)

# Verify features
print("Feature statistics:")
print(df_clean[[
    'Sentiment_Score', 'Avg_Comment_Sentiment', 'Avg_Comment_DistilBERT',
    'Comment_Count', 'Likes_sum', 'Log_Impressions', 'Log_Money_Spent',
    'Sentiment_Interaction', 'Ad_Word_Count', 'Ad_Readability'
]].describe())

# Handle NaN in text
df_clean['Ad_Text'] = df_clean['Ad_Text'].fillna('')
df_clean['Product_Description'] = df_clean['Product_Description'].fillna('')
comments_df['Comment_Text'] = comments_df['Comment_Text'].fillna('')

# Text preprocessing
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    words = [lemmatizer.lemmatize(word) for word in text.split() if len(word) > 2 and word not in {'http', 'www', 'com'}]
    return ' '.join(words) if words else 'neutral'

df_clean['Ad_Text'] = df_clean['Ad_Text'].apply(preprocess_text)
df_clean['Product_Description'] = df_clean['Product_Description'].apply(preprocess_text)
comments_df['Comment_Text'] = comments_df['Comment_Text'].apply(preprocess_text)

# Filter low-content comments
comments_df = comments_df[comments_df['Comment_Text'].str.len() >= 5]
comments_concat = comments_df.groupby('Campaign_ID')['Comment_Text'].apply(lambda x: ' '.join(x)).reset_index()

# Extract TF-IDF features
all_text = df_clean['Ad_Text'] + ' ' + df_clean['Product_Description'] + ' ' + comments_concat['Comment_Text'].reindex(df_clean.index, fill_value='neutral')
tfidf = TfidfVectorizer(max_features=100, stop_words='english', ngram_range=(1, 2))
text_tfidf = tfidf.fit_transform(all_text)

# Apply Truncated SVD
svd = TruncatedSVD(n_components=50, random_state=42)
text_tfidf_reduced = svd.fit_transform(text_tfidf)
print("TF-IDF feature variance ratio:", sum(svd.explained_variance_ratio_))

# One-hot encode categorical features
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
categorical_features = encoder.fit_transform(df_clean[['Platform', 'Category', 'Sentiment_Label']])
categorical_feature_names = encoder.get_feature_names_out(['Platform', 'Category', 'Sentiment_Label'])

# Scale numerical features
scaler = StandardScaler()
numerical_features = scaler.fit_transform(df_clean[[
    'Sentiment_Score', 'Avg_Comment_Sentiment', 'Avg_Comment_DistilBERT',
    'Comment_Count', 'Log_Likes_sum', 'Log_Impressions', 'Log_Money_Spent',
    'Sentiment_Interaction', 'Ad_Word_Count', 'Ad_Readability', 'Ad_DistilBERT', 'Desc_DistilBERT'
]])

# Combine features
X = np.hstack([text_tfidf_reduced, numerical_features, categorical_features])
y = np.log1p(df_clean['Conversions'].values)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Simplified Optuna tuning with correct early stopping
def objective_lgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05),
        'max_depth': trial.suggest_int('max_depth', 3, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 5, 20),
        'random_state': 42,
        'force_col_wise': True
    }
    model = lgb.LGBMRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='rmse',
        callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=False)]
    )
    return -cross_val_score(model, X_train, y_train, cv=3, scoring='r2').mean()

def objective_cb(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05),
        'depth': trial.suggest_int('depth', 3, 5),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 5, 20),
        'random_seed': 42
    }
    model = cb.CatBoostRegressor(**params, verbose=0)
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        early_stopping_rounds=10,
        verbose=0
    )
    return -cross_val_score(model, X_train, y_train, cv=3, scoring='r2').mean()

# Optimize models
study_lgb = optuna.create_study(direction='minimize')
study_lgb.optimize(objective_lgb, n_trials=3)
lgb_model = lgb.LGBMRegressor(**study_lgb.best_params)

study_cb = optuna.create_study(direction='minimize')
study_cb.optimize(objective_cb, n_trials=3)
cb_model = cb.CatBoostRegressor(**study_cb.best_params, verbose=0)

# Simplified Neural Network
nn_model = Sequential([
    Dense(32, activation='relu', input_dim=X_train.shape[1]),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)
])
nn_model.compile(optimizer='adam', loss='mse')
nn_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0, validation_split=0.2)

# Train base models
lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=False)]
)
cb_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=10,
    verbose=0
)
lgb_pred_train = lgb_model.predict(X_train)
lgb_pred_test = lgb_model.predict(X_test)
cb_pred_train = cb_model.predict(X_train)
cb_pred_test = cb_model.predict(X_test)
nn_pred_train = nn_model.predict(X_train, verbose=0).flatten()
nn_pred_test = nn_model.predict(X_test, verbose=0).flatten()

# Stack predictions
stacked_train = np.column_stack((lgb_pred_train, cb_pred_train, nn_pred_train))
stacked_test = np.column_stack((lgb_pred_test, cb_pred_test, nn_pred_test))

# Meta-learner
meta_model = Ridge(alpha=1.0)
meta_model.fit(stacked_train, y_train)
y_pred = meta_model.predict(stacked_test)

# Evaluate
y_pred_back = np.expm1(y_pred)
y_test_back = np.expm1(y_test)
mse = mean_squared_error(y_test_back, y_pred_back)
mae = mean_absolute_error(y_test_back, y_pred_back)
r2 = r2_score(y_test_back, y_pred_back)

print(f"Stacked Ensemble Model Performance (Conversions):")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Mean Absolute Error: {mae:.2f}")
print(f"R² Score: {r2:.4f}")

# Cross-validation
stacked_scores = cross_val_score(meta_model, stacked_train, y_train, cv=5, scoring='r2')
print(f"Cross-Validated R²: {stacked_scores.mean():.4f} ± {stacked_scores.std():.4f}")

# Feature importance via SHAP
explainer = shap.TreeExplainer(lgb_model)
shap_values = explainer.shap_values(X_train)
feature_names = (
    [f'text_tfidf_svd_{i}' for i in range(text_tfidf_reduced.shape[1])] +
    ['Sentiment_Score', 'Avg_Comment_Sentiment', 'Avg_Comment_DistilBERT',
     'Comment_Count', 'Log_Likes_sum', 'Log_Impressions', 'Log_Money_Spent',
     'Sentiment_Interaction', 'Ad_Word_Count', 'Ad_Readability', 'Ad_DistilBERT', 'Desc_DistilBERT'] +
    list(categorical_feature_names)
)
shap_importance = np.abs(shap_values).mean(axis=0)
feature_importance_df = pd.DataFrame({'Feature': feature_names[:len(shap_importance)], 'Importance': shap_importance})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False).head(10)

print("\nTop 10 Most Important Features (SHAP):")
print(feature_importance_df)

# Visualize feature importance
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
plt.title('Top 10 Feature Importances for Conversions Prediction (SHAP)')
plt.tight_layout()
plt.savefig('/kaggle/working/stacked_conversions_feature_importance.png')
plt.close()

# Predict for all campaigns
stacked_all = np.column_stack((
    lgb_model.predict(X),
    cb_model.predict(X),
    nn_model.predict(X, verbose=0).flatten()
))
df_clean['Predicted_Conversions'] = np.expm1(meta_model.predict(stacked_all))
df_clean[['Campaign_ID', 'Ad_Text', 'Product_Description', 'Conversions', 'Predicted_Conversions',
          'Sentiment_Score', 'Avg_Comment_Sentiment', 'Avg_Comment_DistilBERT']].to_csv(
    '/kaggle/working/stacked_conversions_success.csv', index=False
)
print(f"Campaign success predictions saved to /kaggle/working/stacked_conversions_success.csv")

# Display sample predictions
print("\nSample Campaign Success Predictions:")
print(df_clean[['Campaign_ID', 'Ad_Text', 'Conversions', 'Predicted_Conversions', 'Sentiment_Score', 'Avg_Comment_Sentiment']].head())

# Debug comment generation
print("\nComment_Count debug:")
print(comment_agg['Comment_Count'].describe())
print("Sample comments_df counts:", comments_df.groupby('Campaign_ID').size().describe())

Preparing data for fine-tuning DistilBERT...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

[LightGBM] [Info] Total Bins 15293
[LightGBM] [Info] Number of data points in the train set: 1067, number of used features: 71
[LightGBM] [Info] Start training from score 7.151227
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

[I 2025-05-14 11:37:43,838] Trial 2 finished with value: -0.7562649379776921 and parameters: {'n_estimators': 148, 'learning_rate': 0.03102514696348075, 'max_depth': 4, 'reg_lambda': 5.9970064250419055}. Best is trial 2 with value: -0.7562649379776921.
[I 2025-05-14 11:37:43,839] A new study created in memory with name: no-name-7f6fedab-0337-443e-9f23-e2db4840c1fb


[LightGBM] [Info] Total Bins 15293
[LightGBM] [Info] Number of data points in the train set: 1067, number of used features: 71
[LightGBM] [Info] Start training from score 7.151227
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

[I 2025-05-14 11:37:45,174] Trial 0 finished with value: -0.6669966361208696 and parameters: {'iterations': 151, 'learning_rate': 0.011583386542681389, 'depth': 3, 'l2_leaf_reg': 12.189204375623866}. Best is trial 0 with value: -0.6669966361208696.
[I 2025-05-14 11:37:47,268] Trial 1 finished with value: -0.712741973949479 and parameters: {'iterations': 113, 'learning_rate': 0.028717026612488046, 'depth': 5, 'l2_leaf_reg': 15.819095937172934}. Best is trial 1 with value: -0.712741973949479.
[I 2025-05-14 11:37:49,343] Trial 2 finished with value: -0.7497089894067783 and parameters: {'iterations': 173, 'learning_rate': 0.026349714813835474, 'depth': 4, 'l2_leaf_reg': 10.796800349718822}. Best is trial 2 with value: -0.7497089894067783.
Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000729 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15323
[LightGBM] [Info] Number of data points in the train set: 1600, number of used features: 71
[LightGBM] [Info] Start training from score 7.171207
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,